<a href="https://colab.research.google.com/github/TahaRazzaq/Detecting-Political-Bias-in-Urdu-News/blob/main/Political_Bias_in_Urdu_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import glob
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



## Trying NewsPaper

In [3]:
!pip install newspaper3k #important

     |████████████████████████████████| 215kB 2.7MB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 7.4MB 15.3MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=f5c275b76f8b0e88c5b88233e82c33444b2413152a0accede1bf1e3233435283
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398406 sha256=56fae87943c60ebe085a5a41653f47f44ee2b7e8a01f645328db271ed568de90
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13538 sha256=9eca9067647fd9932ef0525f276a269cd7f2bed6127efa71db4fd8c8ec28c6e5
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c77

In [4]:
import newspaper
from newspaper import Article 
import requests
from bs4 import BeautifulSoup

In [6]:

def read_url(url):
  toi_article = Article(url, language="ar") # ar for Arabic but can be used for Urdu 

  return toi_article
  #To download the article 

  # toi_article.download() 



In [7]:
urls = ["https://www.express.pk", "https://jang.com.pk/category/mera-shahar/lahore", 'https://www.nawaiwaqt.com.pk/lahore', 'https://dailypakistan.com.pk/', 'https://mashriqtv.pk/']
links = []
for url in urls:
  response = requests.get(url)
  data = response.text
  soup = BeautifulSoup(data, 'lxml')
  link = []


  for link in soup.find_all('a'):
    link_url = link.get('href')

    if link_url is not None and link_url.startswith('http'):
      links.append(link_url + '\n')
  

In [12]:
domains = [x.split("/")[2] for x in urls]
# domains

In [13]:
useful_links = []

for link in links:
  if len(link) >= 39:
    split = link.split("/")
    if split[2] in domains:
      useful_links.append(link)

useful_links[:5]

['https://www.express.pk/story/2175840/1\n',
 'https://www.express.pk/story/2175777/1\n',
 'https://www.express.pk/story/2175491/1\n',
 'https://www.express.pk/story/2175613/9812\n',
 'https://www.express.pk/story/2175423/508\n']

In [15]:
useful_links = [i[:-1] for i in useful_links]

In [17]:
articles = []
art_text = []
actual_links = []
token_list = []
f = open("/content/drive/MyDrive/DRP_Dr.Agha/archive/stopwords-ur.txt", "r")
stop_words = [x for x in f]
# stop_words = f.readline()
# stop_words = f.readline()
stop_words = [x[:-1] for x in stop_words]
for link in useful_links:
    art = read_url(link)
  # try:
    art.download()
    try:
      art.parse()
      art.nlp()
      temp_text = art.text
      articles.append(art)
      art_text.append(temp_text)
      actual_links.append(link)
    except:
      pass
   


In [18]:
df = pd.DataFrame()
df['Link'] = actual_links
df['Text'] = art_text
df['Articles'] = articles

In [19]:
df.head()

,Link,Text,Articles
0,https://www.express.pk/story/2175722/1,کراچی: پیپلز پارٹی کے چیئرمین بلاول بھٹو زردار...,<newspaper.article.Article object at 0x7fdaec5...
1,https://www.express.pk/videos/2175501/khair-e-...,,<newspaper.article.Article object at 0x7fdaec4...
2,https://www.express.pk/videos/2175500/khabarda...,,<newspaper.article.Article object at 0x7fdaecf...
3,https://www.express.pk/videos/2175500/khabarda...,,<newspaper.article.Article object at 0x7fdaed6...
4,https://www.express.pk/videos/2175390/kal-tak-...,,<newspaper.article.Article object at 0x7fdaec4...


In [20]:

nan_value = float("NaN")
df.replace("", nan_value, inplace=True)

df.dropna(subset = ["Text"], inplace=True)


In [21]:
df = df.drop_duplicates(subset='Text', keep="first")
df.head()

,Link,Text,Articles
0,https://www.express.pk/story/2175722/1,کراچی: پیپلز پارٹی کے چیئرمین بلاول بھٹو زردار...,<newspaper.article.Article object at 0x7fdaec5...
18,https://www.express.pk/story/2175642/1,اسلام آباد: وفاقی وزیر برائے اطلاعات فواد چوہد...,<newspaper.article.Article object at 0x7fdaec4...
23,https://www.express.pk/story/2175589/1,اسلام آباد: پاکستان مسلم لیگ (ن) کے رہنما احسن...,<newspaper.article.Article object at 0x7fdaec4...
25,https://www.express.pk/story/2175721/1,تحریری درخواست اور ثبوت موصول ہونے کے بعد مقدم...,<newspaper.article.Article object at 0x7fdaec4...
27,https://www.express.pk/story/2175570/1,کراچی: سندھ ہائیکورٹ نے دودھ کی قیمتوں میں ہوش...,<newspaper.article.Article object at 0x7fdaec4...


In [22]:
len(df.index)

153

##### **Tokenize**

In [23]:
def removing_punct(text):
  word_tok = word_tokenize(text)
  punctuations = '''۔،!()-[]{};:'"\,<>./?@#$%^&*_~'''
  tokens = []
  for token in word_tok:
    if (token in punctuations):
      continue
    tokens.append(token)
  return tokens




##### **Removing stop words**

In [ ]:
f = open("/content/drive/MyDrive/DRP_Dr.Agha/archive/stopwords-ur.txt", "r")
stop_words = [x for x in f]
stop_words = [x[:-1] for x in stop_words]
word = 'کہتے\n'
word[:-1] in stop_words

True

In [24]:
def removing_stopwords(split_tokens):
  f = open("/content/drive/MyDrive/DRP_Dr.Agha/archive/stopwords-ur.txt", "r")
  stop_words = [x for x in f]
  stop_words = [x[:-1] for x in stop_words]
  final_tokens = []
  for token in split_tokens:
    if (token in stop_words):
      continue
    final_tokens.append(token)
  return final_tokens

In [25]:
split_tokens = [removing_punct(df['Text'][i]) for i in list(df['Text'].index)]
final_tokens = [removing_stopwords(i) for i in split_tokens]

In [26]:
df['Tokens'] = final_tokens
df.head()

,Link,Text,Articles,Tokens
0,https://www.express.pk/story/2175722/1,کراچی: پیپلز پارٹی کے چیئرمین بلاول بھٹو زردار...,<newspaper.article.Article object at 0x7fdaec5...,"[کراچی, پیپلز, پارٹی, چیئرمین, بلاول, بھٹو, زر..."
18,https://www.express.pk/story/2175642/1,اسلام آباد: وفاقی وزیر برائے اطلاعات فواد چوہد...,<newspaper.article.Article object at 0x7fdaec4...,"[اسلام, آباد, وفاقی, وزیر, برائے, اطلاعات, فوا..."
23,https://www.express.pk/story/2175589/1,اسلام آباد: پاکستان مسلم لیگ (ن) کے رہنما احسن...,<newspaper.article.Article object at 0x7fdaec4...,"[اسلام, آباد, پاکستان, مسلم, لیگ, ن, رہنما, اح..."
25,https://www.express.pk/story/2175721/1,تحریری درخواست اور ثبوت موصول ہونے کے بعد مقدم...,<newspaper.article.Article object at 0x7fdaec4...,"[تحریری, درخواست, ثبوت, موصول, ہونے, بعد, مقدم..."
27,https://www.express.pk/story/2175570/1,کراچی: سندھ ہائیکورٹ نے دودھ کی قیمتوں میں ہوش...,<newspaper.article.Article object at 0x7fdaec4...,"[کراچی, سندھ, ہائیکورٹ, نے, دودھ, قیمتوں, میں,..."


In [ ]:
df.to_csv('/content/drive/MyDrive/DRP_Dr.Agha/Newpaper_Articles_1.csv')

In [27]:

df = pd.read_csv('/content/drive/MyDrive/DRP_Dr.Agha/Newpaper_Articles', delimiter=',')
df.head()

,Unnamed: 0,Link,Text,Articles,Tokens
0,0,https://www.express.pk/story/2162926/9812,واشنگٹن: ایک نئی تحقیق کے مطابق رات میں ایک گھ...,<newspaper.article.Article object at 0x7fba119...,"['واشنگٹن', 'نئی', 'تحقیق', 'مطابق', 'رات', 'م..."
1,1,https://www.express.pk/story/2162934/509,قاہرہ: اہرام مصر، فرعون اور ممیوں سے جڑی اساطی...,<newspaper.article.Article object at 0x7fba123...,"['قاہرہ', 'اہرام', 'مصر،', 'فرعون', 'ممیوں', '..."
2,2,https://www.express.pk/story/2162937/10,جکارتا: انڈونیشیا میں طوفانی بارشوں، سیلاب اور...,<newspaper.article.Article object at 0x7fba120...,"['جکارتا', 'انڈونیشیا', 'میں', 'طوفانی', 'بارش..."
3,4,https://www.express.pk/story/2162917/1,اسلام آباد: این سی او سی نے بین الصوبائی ٹرانس...,<newspaper.article.Article object at 0x7fba123...,"['اسلام', 'آباد', 'این', 'سی', 'او', 'سی', 'نے..."
4,5,https://www.express.pk/story/2162910/10,ماسکو: روس میں اوپری منزل میں آگ لگ جانے کے ب...,<newspaper.article.Article object at 0x7fba11a...,"['ماسکو', 'روس', 'میں', 'اوپری', 'منزل', 'میں'..."


In [29]:
df.drop(columns='Unnamed: 0',axis = 1)

,Link,Text,Articles,Tokens
0,https://www.express.pk/story/2162926/9812,واشنگٹن: ایک نئی تحقیق کے مطابق رات میں ایک گھ...,<newspaper.article.Article object at 0x7fba119...,"['واشنگٹن', 'نئی', 'تحقیق', 'مطابق', 'رات', 'م..."
1,https://www.express.pk/story/2162934/509,قاہرہ: اہرام مصر، فرعون اور ممیوں سے جڑی اساطی...,<newspaper.article.Article object at 0x7fba123...,"['قاہرہ', 'اہرام', 'مصر،', 'فرعون', 'ممیوں', '..."
2,https://www.express.pk/story/2162937/10,جکارتا: انڈونیشیا میں طوفانی بارشوں، سیلاب اور...,<newspaper.article.Article object at 0x7fba120...,"['جکارتا', 'انڈونیشیا', 'میں', 'طوفانی', 'بارش..."
3,https://www.express.pk/story/2162917/1,اسلام آباد: این سی او سی نے بین الصوبائی ٹرانس...,<newspaper.article.Article object at 0x7fba123...,"['اسلام', 'آباد', 'این', 'سی', 'او', 'سی', 'نے..."
4,https://www.express.pk/story/2162910/10,ماسکو: روس میں اوپری منزل میں آگ لگ جانے کے ب...,<newspaper.article.Article object at 0x7fba11a...,"['ماسکو', 'روس', 'میں', 'اوپری', 'منزل', 'میں'..."
...,...,...,...,...
181,https://dailypakistan.com.pk/03-Apr-2021/1271829,سپرسانک کمپنی کی امریکی حکام کیلئے مسافر طیارہ...,<newspaper.article.Article object at 0x7fba113...,"['سپرسانک', 'کمپنی', 'امریکی', 'حکام', 'مسافر'..."
182,https://dailypakistan.com.pk/28-Mar-2021/1269166,’ہم تو پاکستان میں دفتر کھول ہی نہیں رہے‘ فیس ...,<newspaper.article.Article object at 0x7fba113...,"['’', 'ہم', 'پاکستان', 'میں', 'دفتر', 'کھول', ..."
183,https://dailypakistan.com.pk/27-Mar-2021/1268694,ویوو نے Zeissکے اشتراک سے نیا فون X60 Pro پاکس...,<newspaper.article.Article object at 0x7fba113...,"['ویوو', 'نے', 'Zeissکے', 'اشتراک', 'سے', 'نیا..."
184,https://dailypakistan.com.pk/30-Mar-2021/1270098,آرمی چیف کا راولپنڈی میں لاجسٹک تنصیبات کادورہ...,<newspaper.article.Article object at 0x7fba113...,"['آرمی', 'چیف', 'کا', 'راولپنڈی', 'میں', 'لاجس..."


In [30]:
toks = [i.split("', '") for i in df.Tokens]

In [31]:
for i in range(len(toks)):
  toks[i][0] = toks[i][0][2:]

In [32]:
df['Tokens'] = toks

In [33]:
df.head()

,Unnamed: 0,Link,Text,Articles,Tokens
0,0,https://www.express.pk/story/2162926/9812,واشنگٹن: ایک نئی تحقیق کے مطابق رات میں ایک گھ...,<newspaper.article.Article object at 0x7fba119...,"[واشنگٹن, نئی, تحقیق, مطابق, رات, میں, گھنٹہ, ..."
1,1,https://www.express.pk/story/2162934/509,قاہرہ: اہرام مصر، فرعون اور ممیوں سے جڑی اساطی...,<newspaper.article.Article object at 0x7fba123...,"[قاہرہ, اہرام, مصر،, فرعون, ممیوں, سے, جڑی, اس..."
2,2,https://www.express.pk/story/2162937/10,جکارتا: انڈونیشیا میں طوفانی بارشوں، سیلاب اور...,<newspaper.article.Article object at 0x7fba120...,"[جکارتا, انڈونیشیا, میں, طوفانی, بارشوں،, سیلا..."
3,4,https://www.express.pk/story/2162917/1,اسلام آباد: این سی او سی نے بین الصوبائی ٹرانس...,<newspaper.article.Article object at 0x7fba123...,"[اسلام, آباد, این, سی, او, سی, نے, بین, الصوبا..."
4,5,https://www.express.pk/story/2162910/10,ماسکو: روس میں اوپری منزل میں آگ لگ جانے کے ب...,<newspaper.article.Article object at 0x7fba11a...,"[ماسکو, روس, میں, اوپری, منزل, میں, آگ, لگ, ج..."


In [34]:
dictionary = []
dictionary = [dictionary + df['Tokens'][i] for i in list(df['Tokens'].index)]



In [35]:
ner = {}
f = open("/content/drive/MyDrive/DRP_Dr.Agha/mk-pucit.txt", "r")
lines = [x for x in f]
w = [lines[i].split("\t") for i in range(len(lines))]
for i in w:
  ner[i[0]] = i[1][:-1]


In [36]:
tokens  = df['Tokens']
for x in tokens:
  print(x)
  break

['واشنگٹن', 'نئی', 'تحقیق', 'مطابق', 'رات', 'میں', 'گھنٹہ', 'اضافی', 'نیند', 'سے', 'کورونا', 'میں', 'مبتلا', 'ہونے', 'خطرات', 'کو', '12', 'فیصد', 'کم', 'کیا', 'جاسکتا', 'ہے۔', 'طبی', 'جریدے', '’', 'بی', 'ایم', 'جے', 'نیوٹریشن،', 'پروینشن', 'اینڈ', 'ہیلتھ', '‘', 'میں', 'حال', 'میں', 'شائع', 'مطالعے', 'سے', 'کورونا', 'ہونے', 'والی', 'تحقیق', 'میں', 'نئی', 'پیش', 'رفت', 'سامنے', 'آئی', 'ہے۔', 'اس', 'مطالعے', 'میں', 'ایسے', '2800', 'سے', 'زائد', 'فرنٹ', 'لائن', 'ہیلتھ', 'ورکرز', 'کو', 'شامل', 'کیا', 'گیا', 'جو', 'اس', 'وبا', 'آغاز', 'سے', 'تاحال', 'کورونا', 'کا', 'سامنا', 'کر', 'ہیں۔', 'تحقیق', 'میں', 'شامل', 'جن', 'ہیلتھ', 'کیئر', 'ورکرز', 'کو', 'رات', 'میں', 'سونے', 'لیے', 'گھنٹہ', 'اضافی', 'وقت', 'دیا', 'گیا', 'ان', 'میں', 'کووڈ', '19', 'میں', 'مبتلا', 'ہونے', 'کا', 'خطرہ', '12', 'فیصد', 'کم', 'ہوگیا', 'جب', 'وہ', 'ورکرز', 'جو', 'گہری', 'نیند', 'سونے', 'میں', 'ناکام', 'ان', 'میں', 'وائرس', 'سے', 'متاثر', 'ہونے', 'خطرات', 'بڑھ', 'گئے۔', 'اس', 'نئی', 'تحقیق', 'بارے', 'میں', '’', 'اوہایو

In [37]:

tokens  = df['Tokens']
ner_tokens = []
ner_list = list(ner.keys())
for toks in tokens:
  ner_toks = []
  for i in toks:
    if (i in ner_list):
      ner_toks.append(ner[i])
    else:
      ner_toks.append("Other")
  ner_tokens.append(ner_toks)


In [ ]:
np.unique(ner_tokens[93])
# 'Organization' in un

array(['Location', 'Organization', 'Other', 'Person'], dtype='<U12')

In [38]:

df['NER_Tokens'] = ner_tokens
df.head()

,Unnamed: 0,Link,Text,Articles,Tokens,NER_Tokens
0,0,https://www.express.pk/story/2162926/9812,واشنگٹن: ایک نئی تحقیق کے مطابق رات میں ایک گھ...,<newspaper.article.Article object at 0x7fba119...,"[واشنگٹن, نئی, تحقیق, مطابق, رات, میں, گھنٹہ, ...","[Location, Other, Other, Other, Other, Other, ..."
1,1,https://www.express.pk/story/2162934/509,قاہرہ: اہرام مصر، فرعون اور ممیوں سے جڑی اساطی...,<newspaper.article.Article object at 0x7fba123...,"[قاہرہ, اہرام, مصر،, فرعون, ممیوں, سے, جڑی, اس...","[Location, Other, Other, Person, Other, Other,..."
2,2,https://www.express.pk/story/2162937/10,جکارتا: انڈونیشیا میں طوفانی بارشوں، سیلاب اور...,<newspaper.article.Article object at 0x7fba120...,"[جکارتا, انڈونیشیا, میں, طوفانی, بارشوں،, سیلا...","[Location, Other, Other, Other, Other, Other, ..."
3,4,https://www.express.pk/story/2162917/1,اسلام آباد: این سی او سی نے بین الصوبائی ٹرانس...,<newspaper.article.Article object at 0x7fba123...,"[اسلام, آباد, این, سی, او, سی, نے, بین, الصوبا...","[Location, Location, Organization, Other, Pers..."
4,5,https://www.express.pk/story/2162910/10,ماسکو: روس میں اوپری منزل میں آگ لگ جانے کے ب...,<newspaper.article.Article object at 0x7fba11a...,"[ماسکو, روس, میں, اوپری, منزل, میں, آگ, لگ, ج...","[Other, Other, Other, Other, Other, Other, Oth..."


In [39]:

counts = [[],[],[],[]]
ls = ['Location', 'Organization', 'Other', 'Person']
for toks in ner_tokens:
  for tag in range(len(ls)):
    indices = [i for i, x in enumerate(toks) if x == ls[tag]]
    counts[tag].append(len(indices))


In [40]:
for x in range(len(ls)):
  df[ls[x]] = counts[x]

df.head()

,Unnamed: 0,Link,Text,Articles,Tokens,NER_Tokens,Location,Organization,Other,Person
0,0,https://www.express.pk/story/2162926/9812,واشنگٹن: ایک نئی تحقیق کے مطابق رات میں ایک گھ...,<newspaper.article.Article object at 0x7fba119...,"[واشنگٹن, نئی, تحقیق, مطابق, رات, میں, گھنٹہ, ...","[Location, Other, Other, Other, Other, Other, ...",7,4,321,1
1,1,https://www.express.pk/story/2162934/509,قاہرہ: اہرام مصر، فرعون اور ممیوں سے جڑی اساطی...,<newspaper.article.Article object at 0x7fba123...,"[قاہرہ, اہرام, مصر،, فرعون, ممیوں, سے, جڑی, اس...","[Location, Other, Other, Person, Other, Other,...",10,1,227,5
2,2,https://www.express.pk/story/2162937/10,جکارتا: انڈونیشیا میں طوفانی بارشوں، سیلاب اور...,<newspaper.article.Article object at 0x7fba120...,"[جکارتا, انڈونیشیا, میں, طوفانی, بارشوں،, سیلا...","[Location, Other, Other, Other, Other, Other, ...",3,0,182,0
3,4,https://www.express.pk/story/2162917/1,اسلام آباد: این سی او سی نے بین الصوبائی ٹرانس...,<newspaper.article.Article object at 0x7fba123...,"[اسلام, آباد, این, سی, او, سی, نے, بین, الصوبا...","[Location, Location, Organization, Other, Pers...",2,4,143,3
4,5,https://www.express.pk/story/2162910/10,ماسکو: روس میں اوپری منزل میں آگ لگ جانے کے ب...,<newspaper.article.Article object at 0x7fba11a...,"[ماسکو, روس, میں, اوپری, منزل, میں, آگ, لگ, ج...","[Other, Other, Other, Other, Other, Other, Oth...",0,1,138,0


In [41]:
ls = ['کوئٹہ','لاہور','وزیراعظم','اسلام','کراچی','صوبے','پاکستانی','سندھ','کشمیر','آرمی',]
new_df_2 = []
for i in df.index:
  if df['Tokens'][i][0] in ls:
    try:
      new_df_2.append(df.iloc[i])
    except:
      pass

In [42]:
# new_df_2 = pd.DataFrame(new_df_2)
new_df_2 = df
new_df_2 = new_df_2[new_df_2['Organization'] > 0]
new_df_2.reset_index()
new_df_2.head()


,Unnamed: 0,Link,Text,Articles,Tokens,NER_Tokens,Location,Organization,Other,Person
0,0,https://www.express.pk/story/2162926/9812,واشنگٹن: ایک نئی تحقیق کے مطابق رات میں ایک گھ...,<newspaper.article.Article object at 0x7fba119...,"[واشنگٹن, نئی, تحقیق, مطابق, رات, میں, گھنٹہ, ...","[Location, Other, Other, Other, Other, Other, ...",7,4,321,1
1,1,https://www.express.pk/story/2162934/509,قاہرہ: اہرام مصر، فرعون اور ممیوں سے جڑی اساطی...,<newspaper.article.Article object at 0x7fba123...,"[قاہرہ, اہرام, مصر،, فرعون, ممیوں, سے, جڑی, اس...","[Location, Other, Other, Person, Other, Other,...",10,1,227,5
3,4,https://www.express.pk/story/2162917/1,اسلام آباد: این سی او سی نے بین الصوبائی ٹرانس...,<newspaper.article.Article object at 0x7fba123...,"[اسلام, آباد, این, سی, او, سی, نے, بین, الصوبا...","[Location, Location, Organization, Other, Pers...",2,4,143,3
4,5,https://www.express.pk/story/2162910/10,ماسکو: روس میں اوپری منزل میں آگ لگ جانے کے ب...,<newspaper.article.Article object at 0x7fba11a...,"[ماسکو, روس, میں, اوپری, منزل, میں, آگ, لگ, ج...","[Other, Other, Other, Other, Other, Other, Oth...",0,1,138,0
5,6,https://www.express.pk/story/2162900/16,ڈھاکا: پاکستان نے تین ملکی ٹی ٹوئنٹی بلائنڈکرک...,<newspaper.article.Article object at 0x7fba11a...,"[ڈھاکا, پاکستان, نے, تین, ملکی, ٹی, ٹوئنٹی, بل...","[Organization, Location, Other, Other, Other, ...",21,4,151,9


In [43]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Newpaper_Articles_combined').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# rows = worksheet.get_all_values()
df = pd.DataFrame.from_records(rows[1:],columns=rows[0])

In [44]:
# x = pd.read_csv('/content/drive/MyDrive/DRP_Dr.Agha/Newpaper_Articles_combined.gsheet')
df.columns

Index(['', 'Link', 'Text', 'Articles', 'Tokens', 'NER_Tokens', 'Location',
       'Organization', 'Other', 'Person', 'Taha's Label', 'Raufs Labels'],
      dtype='object')

In [45]:
# df = pd.read_csv('/content/drive/MyDrive/DRP_Dr.Agha/Newpaper_Articles_combined.csv', delimiter=',')
df.drop(df.columns[0],axis = 1, inplace=True)
df.head()

,Link,Text,Articles,Tokens,NER_Tokens,Location,Organization,Other,Person,Taha's Label,Raufs Labels
0,https://www.express.pk/story/2168914/1,لاہور: کالعدم ٹی ایل پی کے سربراہ سعد رضوی سے ...,<newspaper.article.Article object at 0x7f363be...,"['لاہور', 'کالعدم', 'ٹی', 'ایل', 'پی', 'سربراہ...","['Location', 'Organization', 'Organization', '...",22,28,356,28,0,0
1,https://www.express.pk/story/2168932/1,وزیراعظم عمران خان نے جہانگیر ترین گروپ کی بات...,<newspaper.article.Article object at 0x7f363c9...,"['وزیراعظم', 'عمران', 'خان', 'نے', 'جہانگیر', ...","['Other', 'Person', 'Other', 'Other', 'Person'...",1,3,111,7,0,0
2,https://www.express.pk/story/2168360/1,پاکستانی قوم دنیا کی ان پانچ بڑی اقوام میں شام...,<newspaper.article.Article object at 0x7f363be...,"['پاکستانی', 'قوم', 'دنیا', 'ان', 'پانچ', 'بڑی...","['Other', 'Other', 'Other', 'Other', 'Other', ...",43,59,1875,13,0,0
3,https://www.express.pk/story/2168771/1,اسلام آباد: قومی اسمبلی اجلاس کے دوران اپوزیشن...,<newspaper.article.Article object at 0x7f363bd...,"['اسلام', 'آباد', 'قومی', 'اسمبلی', 'اجلاس', '...","['Location', 'Location', 'Other', 'Other', 'Ot...",10,21,390,25,0,0
4,https://dailypakistan.com.pk/17-Jun-2020/1145808,سندھ حکومت تنخواہوں اور پینشن میں کتنا اضافہ ک...,<newspaper.article.Article object at 0x7f363ba...,"['سندھ', 'حکومت', 'تنخواہوں', 'پینشن', 'میں', ...","['Other', 'Other', 'Other', 'Other', 'Other', ...",2,5,427,2,1,1


In [ ]:
# df.drop(df.columns[0], axis = 1, inplace = True)
df = df.append(new_df_2)
df.head()

,Link,Text,Articles,Tokens,NER_Tokens,Location,Organization,Other,Person,Taha's Label,Raufs Labels
0,https://www.express.pk/story/2168914/1,لاہور: کالعدم ٹی ایل پی کے سربراہ سعد رضوی سے ...,<newspaper.article.Article object at 0x7f363be...,"['لاہور', 'کالعدم', 'ٹی', 'ایل', 'پی', 'سربراہ...","['Location', 'Organization', 'Organization', '...",22,28,356,28,0,
1,https://www.express.pk/story/2168932/1,وزیراعظم عمران خان نے جہانگیر ترین گروپ کی بات...,<newspaper.article.Article object at 0x7f363c9...,"['وزیراعظم', 'عمران', 'خان', 'نے', 'جہانگیر', ...","['Other', 'Person', 'Other', 'Other', 'Person'...",1,3,111,7,0,
2,https://www.express.pk/story/2168360/1,پاکستانی قوم دنیا کی ان پانچ بڑی اقوام میں شام...,<newspaper.article.Article object at 0x7f363be...,"['پاکستانی', 'قوم', 'دنیا', 'ان', 'پانچ', 'بڑی...","['Other', 'Other', 'Other', 'Other', 'Other', ...",43,59,1875,13,0,
3,https://www.express.pk/story/2168771/1,اسلام آباد: قومی اسمبلی اجلاس کے دوران اپوزیشن...,<newspaper.article.Article object at 0x7f363bd...,"['اسلام', 'آباد', 'قومی', 'اسمبلی', 'اجلاس', '...","['Location', 'Location', 'Other', 'Other', 'Ot...",10,21,390,25,0,
4,https://dailypakistan.com.pk/17-Jun-2020/1145808,سندھ حکومت تنخواہوں اور پینشن میں کتنا اضافہ ک...,<newspaper.article.Article object at 0x7f363ba...,"['سندھ', 'حکومت', 'تنخواہوں', 'پینشن', 'میں', ...","['Other', 'Other', 'Other', 'Other', 'Other', ...",2,5,427,2,1,


In [ ]:
df.to_csv('/content/drive/MyDrive/DRP_Dr.Agha/Newpaper_Articles_combined_new.csv')

In [48]:
features = ['Location','Organization', 'Other', 'Person']
data = df[features]

In [70]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(data)
lbls = kmeans.labels_

In [52]:

labels = df[(list(df.columns))[-2]]
labels = [int(labels[i]) for i in range(len(labels))]

In [72]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(labels,lbls)
acc

0.64

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=0)

In [107]:
clf_dt = DecisionTreeClassifier().fit(X_train,y_train)
pred = clf_dt.predict(X_test[:-1])
acc = accuracy_score(y_test[:-1],pred)
acc

0.6666666666666666

In [83]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
pred = clf.predict(X_test[:-1])

acc = accuracy_score(y_test[:-1],pred)
acc

0.5555555555555556

In [84]:

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test[:-1])
acc = accuracy_score(y_test[:-1],y_pred)
acc

0.4444444444444444

Testing Pipeline

In [90]:

def transforming_test_art(url):
  art = read_url(url)
  art.download()
  art.parse()
  art.nlp()
  temp_text = art.text

  split_tokens = [removing_punct(temp_text)]
  final_tokens = [removing_stopwords(i) for i in split_tokens]


  ner_tokens = []
  ner_list = list(ner.keys())
  for toks in final_tokens:
    ner_toks = []
    for i in toks:
      if (i in ner_list):
        ner_toks.append(ner[i])
      else:
        ner_toks.append("Other")
    ner_tokens.append(ner_toks)

  counts = [None,None,None,None]
  ls = ['Location', 'Organization', 'Other', 'Person']
  for toks in ner_tokens:
    for tag in range(len(ls)):
      indices = [i for i, x in enumerate(toks) if x == ls[tag]]
      counts[tag] = len(indices)
  return counts

In [ ]:
cnts = transforming_test_art('https://www.express.pk/story/2175840/1/') 

In [119]:
# counts_test = X_test[-1
cnts = X_test.loc[(list(X_test.index))[-3:]]
# cnts = [[i] for i in list(cnts)]
# cnts

In [121]:
test_pred = clf_dt.predict(cnts)
test_pred


array([0, 1, 1])

### UrduHack

In [ ]:
!pip install Urduhack[tf]

In [ ]:
import urduhack
urduhack.download()

90112/89078 [==============================] - 0s 0us/step


In [ ]:
from urduhack import Pipeline
nlp = Pipeline()
text = """
اسلام آباد: این سی او سی نے بین الصوبائی ٹرانسپورٹ ہفتے میں دو دن بند رکھنے کا فیصلہ کرلیا۔

این سی او سی نے جانب سے جاری بیان میں کہا گیاہے کہ ملک بھر میں بین الصوبائی پبلک ٹرانسپورٹ ہفتے میں دو دن بند رکھنے کا فیصلہ کیا ہے، ٹرانسپورٹ ہفتہ اتوار بند رکھی جائے گی۔

اس خبرکوبھی پڑھیں: سندھ حکومت کی وفاق سے بین الصوبائی ٹرانسپورٹ بند کرنے کی سفارش

این سی او سی کے مطابق گڈز ٹرانسپورٹ سروس اور ایمرجنسی ٹرانسپورٹ سروس جاری رہے گی، 70 فیصد مسافروں کے ریلوے سروس ہفتے کے 7 روز جاری رہے گی، بین الصوبائی ٹرانسپورٹ سروس 10 اپریل سے 25 اپریل تک ہفتہ اور اتوار کو بند رہے گی، تاہم ٹرانسپورٹ سے متعلق فیصلے کے لیے نظرثانی اجلاس 20 اپریل کو ہوگا۔

واضح رہے کہ دو روز قبل وزیراعلیٰ سندھ نے ایک اجلاس کی صدارت کرتےہوئے وفاق سے مطالبہ کیا تھا کہ وہ لاک ڈاؤن کے حق میں نہیں لیکن وہ صرف بین الصوبائی ٹرانسپورٹ پر پابندی عائد کرنا چاہتے ہیں تاکہ لوگ ایک صوبے سے دوسرے صوبے میں داخل نہ ہوسکیں۔
"""
doc = nlp(text)

for sentence in doc.sentences:
    print(sentence.text)
    for word in sentence.words:
        print(f"{word.text}\t{word.pos}")

    for token in sentence.tokens:
        print(f"{token.text}\t{token.ner}")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:390: UserWarning: CRF Decoding does not work with KerasTensors in TF2.4. The bug has since been fixed in tensorflow/tensorflow##45534
  return py_builtins.overload_of(f)(*args)


ValueError: ignored

In [ ]:
df['Text'][0]

'اسلام آباد: این سی او سی نے بین الصوبائی ٹرانسپورٹ ہفتے میں دو دن بند رکھنے کا فیصلہ کرلیا۔\n\nاین سی او سی نے جانب سے جاری بیان میں کہا گیاہے کہ ملک بھر میں بین الصوبائی پبلک ٹرانسپورٹ ہفتے میں دو دن بند رکھنے کا فیصلہ کیا ہے، ٹرانسپورٹ ہفتہ اتوار بند رکھی جائے گی۔\n\nاس خبرکوبھی پڑھیں: سندھ حکومت کی وفاق سے بین الصوبائی ٹرانسپورٹ بند کرنے کی سفارش\n\nاین سی او سی کے مطابق گڈز ٹرانسپورٹ سروس اور ایمرجنسی ٹرانسپورٹ سروس جاری رہے گی، 70 فیصد مسافروں کے ریلوے سروس ہفتے کے 7 روز جاری رہے گی، بین الصوبائی ٹرانسپورٹ سروس 10 اپریل سے 25 اپریل تک ہفتہ اور اتوار کو بند رہے گی، تاہم ٹرانسپورٹ سے متعلق فیصلے کے لیے نظرثانی اجلاس 20 اپریل کو ہوگا۔\n\nواضح رہے کہ دو روز قبل وزیراعلیٰ سندھ نے ایک اجلاس کی صدارت کرتےہوئے وفاق سے مطالبہ کیا تھا کہ وہ لاک ڈاؤن کے حق میں نہیں لیکن وہ صرف بین الصوبائی ٹرانسپورٹ پر پابندی عائد کرنا چاہتے ہیں تاکہ لوگ ایک صوبے سے دوسرے صوبے میں داخل نہ ہوسکیں۔'

#### Bi / Tri Grams

In [ ]:
bi_grams = list(nltk.bigrams(tokens))
tri_grams = list(nltk.trigrams(tokens))
print(bi_grams[:5])
tri_grams[:5]

In [ ]:
('اسلام', 'آباد') in bi_grams

In [ ]:
from collections import Counter
uni_gram_counts = Counter(tokens)
bi_gram_counts = Counter(bi_grams)


In [ ]:

prob_bigram = {}
print(uni_gram_counts['آباد'])
total_bigrams = len(bi_grams)
for bigrm in bi_grams:
  cnt = bi_gram_counts[bigrm]
  prob_bigram[bigrm] = cnt/total_bigrams

for k,v in prob_bigram.items():
  if v > 0.01:
    print(k," ",v)

In [ ]:
prob_trigram = {}
print(uni_gram_counts['آباد'])
total_trigrams = len(tri_grams)
for trigrm in tri_grams:
  cnt = bi_gram_counts[trigrm]
  prob_trigram[trigrm] = cnt/total_trigrams

for k,v in prob_trigram.items():
  if v > 0.01:
    print(k," ",v)

In [ ]:

#To extract title 
print("Article's Title:") 
print(toi_article.title) 
print("\n") 
  
#To extract text 
print("Article's Text:") 
print(toi_article.text) 
print("\n") 
  
#To extract summary 
print("Article's Summary:") 
print(toi_article.summary) 
print("\n") 
  
#To extract keywords 
print("Article's Keywords:") 
print(toi_article.keywords) 

### Trying SpaCy

In [ ]:
!pip install spacy

In [ ]:
import spacy

nlp = spacy.blank('ur')


In [ ]:
doc = nlp(toi_article.text)

In [ ]:
doc

In [ ]:
tokens = doc.text.split(" ")

In [ ]:
tokens[:5]

In [ ]:
lexeme = doc.vocab[doc[0].text]
print(lexeme.text, lexeme.orth, lexeme.shape_, lexeme.prefix_, lexeme.suffix_,
            lexeme.is_alpha, lexeme.is_digit, lexeme.is_title, lexeme.lang_)

In [ ]:
toi_article.authors

In [ ]:
toi_article.publish_date
# toi_article.publish_d